In [311]:
from pathlib import Path
import re
import numpy as np
from collections import Counter
import pandas as pd

In [312]:
FILEPATHS = [Path("../Start.py")] + list(Path("..").glob("pages/*.py"))
ALLVPATH = Path("allversion.po")
RENPATH = Path("allversion_renamed.po")

In [313]:
# Define the file abbreviations
abbr = {
    '_sidebar.py': "SDE",
    'Start.py' : "STR",
    'photosynthesis.py': "PHO",
    'method.py': "MTH",
    'model_explain.py': "MDL",
    'first_analysis.py': "FAL",
    'plant_memory.py': "MEM",
    'conclusion.py': "CON",
    'contact.py': "COT",
}

no_rep = [
    'LITERATURE_ONPAGE',
    'LITERATURE',
    'AXIS_NPQ',
    'AXIS_PHIPSII',
    'AXIS_TIME_MIN',
    'SLIDER_LIGHT',
    'SLIDER_DEACTIVATION',
    'SLIDER_PULSES',
    'FLUO',
    'SPINNER',
    'SLIDER_ACTIVATION',
    'AXIS_TIME_S'
]

In [314]:
# Extract the text placeholders
rewrite = False

text_re = '(?<=text\(")[\w]+(?="\))|(?<=markdown_click\(")[\w]+(?=")'
texts = {}
texts_nr = []
all_texts = []
pos = {}

for FILEPATH in FILEPATHS:
    keys=[]
    _keys=[]
    keys_nr=[]
    with open(FILEPATH, "r") as f:
        # _RENPATH = FILEPATH.parent / (str(FILEPATH.stem) + "_renamed" + str(FILEPATH.suffix))
        _RENPATH = FILEPATH
        
        lins = f.readlines()
        _lins = lins.copy()

        for i,lin in enumerate(lins):
            _match = re.search(text_re, lin)
            if _match:
                for _mtch in re.finditer(text_re, lin):
                    if _mtch.group(0) not in no_rep:
                        _new = abbr[FILEPATH.name] + "_" + _mtch.group(0)
                        _lins[i] = re.sub(_mtch.group(0), _new,_lins[i])

                        keys += [_mtch.group(0)]
                    else:
                        _keys += [_mtch.group(0)]
                    # Add instance of found key to the positions
                    pos.setdefault(_mtch.group(0), []).append(f"#: {str(FILEPATH.relative_to('..'))}:{i+1}\n")

    if len(keys) >0:
        texts[FILEPATH.name] = list(set(keys))
        texts_nr += list(set(_keys))
        all_texts += list(set(keys)) + list(set(_keys))

        if rewrite:
            with open(_RENPATH, "w") as f:
                f.writelines(_lins)

texts_nr = list(set(texts_nr))

for k,v in pos.items():
    pos[k] = sorted(list(set(v)))

In [315]:
# Find Placeholders with multiple occurrences
all_texts_strp = [re.sub("|".join([f"^{x}_" for x in abbr.values()]), "", txt) for txt in all_texts]
all_texts_strp_cnt = pd.Series(Counter(all_texts_strp))
all_texts_strp_cnt[all_texts_strp_cnt>1].index

Index(['PROGRAMS_USED', 'LITERATURE_ONPAGE', 'LITERATURE', 'AXIS_TIME_MIN',
       'SLIDER_LIGHT', 'SLIDER_DEACTIVATION', 'AXIS_NPQ', 'AXIS_PHIPSII',
       'SLIDER_PULSES', 'FLUO', 'SPINNER', 'SLIDER_ACTIVATION', 'AXIS_TIME_S'],
      dtype='object')

In [316]:
rewrite = True
line_references = True
add_placeholder_abbr = False

ctr = 0
with open(ALLVPATH, "r") as alv:
    lins = alv.readlines()
    _lins = lins.copy()
    for i, lin in enumerate(lins):
        if re.search("msgid", lin):
            for nam, plcs in texts.items():
                for plc in plcs:
                    if re.search('"'+plc+'"', lin):
                        if add_placeholder_abbr:
                            _lins[i+ctr] = re.sub(plc, abbr[nam] + "_" + plc, lin)
                        if line_references:
                            _lins[i+ctr:i+ctr] = pos[plc]
                            ctr += len(pos[plc])
            if line_references:
                for plc in texts_nr:
                    if re.search('"'+plc+'"', lin):
                        _lins[i+ctr:i+ctr] = pos[plc]
                        ctr += len(pos[plc])


if rewrite:                
    with open(ALLVPATH, "w") as f:
        f.writelines(_lins)       
                